# Fixing initial problems
The original regional averaging code didn't write the 0th element of every array and screwed up the time axis


In [1]:
import cdms2 as cdms
import MV2 as MV
import cdtime,cdutil,genutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import glob
import scipy.stats as stats
# Local solution
# If running remotely, uncomment the following code:
# %%bash
# git clone https://github.com/katemarvel/CMIP5_tools
# import CMIP5_tools as cmip5
import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
%matplotlib inline

In [2]:
central_plains=cdutil.region.domain(longitude=(-105,-92),latitude=(32,46))
southwest=cdutil.region.domain(longitude=(-125,-105),latitude=(32,41))


In [3]:
#sshfs mout to kdm2144@dester.ldeo.columbia.edu:/home/kdm2144/ 
#datadirec="/Users/kmarvel/Documents/DATA/dester/"
datadirec="/Volumes/RAVEN/CMIP6/cmip6rawyr/"
writedirec="/Users/kmarvel/Documents/DATA/dester/regional_averages/"
fixedvardirec="/Users/kmarvel/Documents/DATA/dester/fixedvar/"

In [16]:
experiments=["historical","ssp126","ssp245","ssp370","ssp585"]

In [17]:
#Start again with mrsos
#variables=[x.split("/")[-1] for x in glob.glob(datadirec+"*")]
variables=['mrsos', 'pr', 'prsn', 'tas']

###### LOOP OVER ALL VARIABLES #####
for variable in variables:
    
    modeldirs=glob.glob(datadirec+variable+"/*")
    ### LOOP OVER ALL MODELS
    for direc in modeldirs:
        model=direc.split("/")[-1]
        #print(model)
        allfiles=glob.glob(direc+"/*"+variable+"*")
        #experiments=np.unique([x.split(".")[1] for x in allfiles])
        experiments=["historical","ssp126","ssp245","ssp370","ssp585"]
        ###### LOOP OVER ALL EXPERIMENTS #####
        for experiment in experiments:
            #print(experiment)
            cp_writedirec=writedirec+"central_plains/"+variable+"/"+experiment+"/"
            
            sw_writedirec=writedirec+"southwest/"+variable+"/"+experiment+"/"
            
            allfiles_experiment=glob.glob(direc+"/"+variable+"."+experiment+".*")
            rips=np.unique([x.split(".")[-3] for x in allfiles_experiment])

            landthresh=1
            #Get the land fraction
            landfiles=glob.glob(fixedvardirec+"sftlf*"+model+".*")
            

            if len(landfiles)==1:
                fland=cdms.open(landfiles[0])
                landfrac=fland("sftlf")
                fland.close()
            else:
                #print("can't find land fraction file for", model)
                #print(landfiles)
                continue


            ###### LOOP OVER ALL RIPS #####
            for rip in rips:
                #Open the existing data
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fcpw=cdms.open(cp_writename)    
                CP=fcpw(variable)
                fcpw.close()
                sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fsww=cdms.open(sw_writename)    
                SW = fsww(variable)
                fsww.close()  
                
                
                
                ripfiles=np.sort(glob.glob(direc+"/"+variable+"."+experiment+"."+model+"."+rip+"*"))
                print(variable+"."+experiment+"."+model+"."+rip+"*")
                L=len(ripfiles)
                i=0
                ripfile=ripfiles[i]
                frip=cdms.open(ripfile)
                data=frip(variable)
                frip.close()
                if data.shape[1:]!=landfrac.shape:
                    print ("land mask wrong shape for "+variable+"."+experiment+"."+model+"."+rip)
                    continue
                
                
                landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
               
                CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
                SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
             

                tax=cdms.createAxis(np.arange(L*12))
                tax.designateTime()
                tax.units='months since '+str(fyear)+'-1-1'
                CP.setAxis(0,tax)
                SW.setAxis(0,tax)
                CP.id=variable
                SW.id=variable
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fcpw=cdms.open(cp_writename,"w")    
                fcpw.write(CP)
                fcpw.close()
                sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fsww=cdms.open(sw_writename,"w")    
                fsww.write(SW)
                fsww.close()  

mrsos.historical.BCC-CSM2-MR.r1i1p1f1*


/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3169: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dout = self.data[indx]
/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3201: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mout = _mask[indx]


mrsos.historical.BCC-CSM2-MR.r2i1p1f1*
mrsos.historical.BCC-CSM2-MR.r3i1p1f1*
mrsos.ssp126.BCC-CSM2-MR.r1i1p1f1*
mrsos.ssp245.BCC-CSM2-MR.r1i1p1f1*
mrsos.ssp370.BCC-CSM2-MR.r1i1p1f1*
mrsos.ssp585.BCC-CSM2-MR.r1i1p1f1*
mrsos.historical.BCC-ESM1.r1i1p1f1*
mrsos.historical.BCC-ESM1.r2i1p1f1*
mrsos.historical.BCC-ESM1.r3i1p1f1*
mrsos.historical.CanESM5.r10i1p1f1*
mrsos.historical.CanESM5.r10i1p2f1*
mrsos.historical.CanESM5.r11i1p1f1*
mrsos.historical.CanESM5.r12i1p1f1*
mrsos.historical.CanESM5.r13i1p1f1*
mrsos.historical.CanESM5.r14i1p1f1*
mrsos.historical.CanESM5.r15i1p1f1*
mrsos.historical.CanESM5.r16i1p1f1*
mrsos.historical.CanESM5.r17i1p1f1*
mrsos.historical.CanESM5.r18i1p1f1*
mrsos.historical.CanESM5.r19i1p1f1*
mrsos.historical.CanESM5.r1i1p1f1*
mrsos.historical.CanESM5.r1i1p2f1*
mrsos.historical.CanESM5.r20i1p1f1*
mrsos.historical.CanESM5.r21i1p1f1*
mrsos.historical.CanESM5.r22i1p1f1*
mrsos.historical.CanESM5.r23i1p1f1*
mrsos.historical.CanESM5.r24i1p1f1*
mrsos.historical.CanESM5.r25

mrsos.ssp126.IPSL-CM6A-LR.r1i1p1f1*
mrsos.ssp126.IPSL-CM6A-LR.r2i1p1f1*
mrsos.ssp126.IPSL-CM6A-LR.r3i1p1f1*
mrsos.ssp245.IPSL-CM6A-LR.r1i1p1f1*
mrsos.ssp245.IPSL-CM6A-LR.r2i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r10i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r1i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r2i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r3i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r4i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r5i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r6i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r7i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r8i1p1f1*
mrsos.ssp370.IPSL-CM6A-LR.r9i1p1f1*
mrsos.ssp585.IPSL-CM6A-LR.r1i1p1f1*
mrsos.historical.MIROC-ES2L.r1i1p1f2*
mrsos.historical.MIROC-ES2L.r2i1p1f2*
mrsos.historical.MIROC-ES2L.r3i1p1f2*
mrsos.ssp126.MIROC-ES2L.r1i1p1f2*
mrsos.ssp245.MIROC-ES2L.r1i1p1f2*
mrsos.ssp370.MIROC-ES2L.r1i1p1f2*
mrsos.ssp585.MIROC-ES2L.r1i1p1f2*
mrsos.historical.MIROC6.r10i1p1f1*
mrsos.historical.MIROC6.r1i1p1f1*
mrsos.historical.MIROC6.r2i1p1f1*
mrsos.historical.MIROC6.r3i1p1f1*
mrsos.historical.MIROC6.r4i1p1f1*
mr

pr.ssp245.CNRM-CM6-1.r6i1p1f2*
pr.ssp370.CNRM-CM6-1.r1i1p1f2*
pr.ssp370.CNRM-CM6-1.r2i1p1f2*
pr.ssp370.CNRM-CM6-1.r3i1p1f2*
pr.ssp370.CNRM-CM6-1.r4i1p1f2*
pr.ssp370.CNRM-CM6-1.r5i1p1f2*
pr.ssp370.CNRM-CM6-1.r6i1p1f2*
pr.ssp585.CNRM-CM6-1.r1i1p1f2*
pr.ssp585.CNRM-CM6-1.r2i1p1f2*
pr.ssp585.CNRM-CM6-1.r3i1p1f2*
pr.ssp585.CNRM-CM6-1.r4i1p1f2*
pr.ssp585.CNRM-CM6-1.r5i1p1f2*
pr.ssp585.CNRM-CM6-1.r6i1p1f2*
pr.historical.CNRM-ESM2-1.r1i1p1f2*
pr.historical.CNRM-ESM2-1.r2i1p1f2*
pr.historical.CNRM-ESM2-1.r3i1p1f2*
pr.historical.CNRM-ESM2-1.r4i1p1f2*
pr.historical.CNRM-ESM2-1.r5i1p1f2*
pr.ssp126.CNRM-ESM2-1.r1i1p1f2*
pr.ssp126.CNRM-ESM2-1.r2i1p1f2*
pr.ssp126.CNRM-ESM2-1.r3i1p1f2*
pr.ssp126.CNRM-ESM2-1.r4i1p1f2*
pr.ssp126.CNRM-ESM2-1.r5i1p1f2*
pr.ssp245.CNRM-ESM2-1.r1i1p1f2*
pr.ssp245.CNRM-ESM2-1.r2i1p1f2*
pr.ssp245.CNRM-ESM2-1.r3i1p1f2*
pr.ssp245.CNRM-ESM2-1.r4i1p1f2*
pr.ssp245.CNRM-ESM2-1.r5i1p1f2*
pr.ssp370.CNRM-ESM2-1.r1i1p1f2*
pr.ssp370.CNRM-ESM2-1.r2i1p1f2*
pr.ssp370.CNRM-ESM2-1.r3i1p1f2*
p

prsn.historical.CanESM5.r8i1p2f1*
prsn.historical.CanESM5.r9i1p1f1*
prsn.historical.CanESM5.r9i1p2f1*
prsn.ssp245.CanESM5.r10i1p1f1*
prsn.ssp245.CanESM5.r10i1p2f1*
prsn.ssp245.CanESM5.r1i1p1f1*
prsn.ssp245.CanESM5.r1i1p2f1*
prsn.ssp245.CanESM5.r2i1p1f1*
prsn.ssp245.CanESM5.r2i1p2f1*
prsn.ssp245.CanESM5.r3i1p1f1*
prsn.ssp245.CanESM5.r3i1p2f1*
prsn.ssp245.CanESM5.r4i1p1f1*
prsn.ssp245.CanESM5.r4i1p2f1*
prsn.ssp245.CanESM5.r5i1p1f1*
prsn.ssp245.CanESM5.r5i1p2f1*
prsn.ssp245.CanESM5.r6i1p1f1*
prsn.ssp245.CanESM5.r6i1p2f1*
prsn.ssp245.CanESM5.r7i1p1f1*
prsn.ssp245.CanESM5.r7i1p2f1*
prsn.ssp245.CanESM5.r8i1p1f1*
prsn.ssp245.CanESM5.r8i1p2f1*
prsn.ssp245.CanESM5.r9i1p1f1*
prsn.ssp245.CanESM5.r9i1p2f1*
prsn.ssp370.CanESM5.r10i1p1f1*
prsn.ssp370.CanESM5.r10i1p2f1*
prsn.ssp370.CanESM5.r1i1p1f1*
prsn.ssp370.CanESM5.r1i1p2f1*
prsn.ssp370.CanESM5.r2i1p1f1*
prsn.ssp370.CanESM5.r2i1p2f1*
prsn.ssp370.CanESM5.r3i1p1f1*
prsn.ssp370.CanESM5.r3i1p2f1*
prsn.ssp370.CanESM5.r4i1p1f1*
prsn.ssp370.CanESM5.r4i1

tas.historical.CanESM5.r10i1p2f1*
tas.historical.CanESM5.r11i1p1f1*
tas.historical.CanESM5.r12i1p1f1*
tas.historical.CanESM5.r13i1p1f1*
tas.historical.CanESM5.r14i1p1f1*
tas.historical.CanESM5.r15i1p1f1*
tas.historical.CanESM5.r16i1p1f1*
tas.historical.CanESM5.r17i1p1f1*
tas.historical.CanESM5.r18i1p1f1*
tas.historical.CanESM5.r19i1p1f1*
tas.historical.CanESM5.r1i1p1f1*
tas.historical.CanESM5.r1i1p2f1*
tas.historical.CanESM5.r20i1p1f1*
tas.historical.CanESM5.r21i1p1f1*
tas.historical.CanESM5.r22i1p1f1*
tas.historical.CanESM5.r23i1p1f1*
tas.historical.CanESM5.r24i1p1f1*
tas.historical.CanESM5.r25i1p1f1*
tas.historical.CanESM5.r2i1p1f1*
tas.historical.CanESM5.r2i1p2f1*
tas.historical.CanESM5.r3i1p1f1*
tas.historical.CanESM5.r3i1p2f1*
tas.historical.CanESM5.r4i1p1f1*
tas.historical.CanESM5.r4i1p2f1*
tas.historical.CanESM5.r5i1p1f1*
tas.historical.CanESM5.r5i1p2f1*
tas.historical.CanESM5.r6i1p1f1*
tas.historical.CanESM5.r6i1p2f1*
tas.historical.CanESM5.r7i1p1f1*
tas.historical.CanESM5.r7i1

tas.historical.IPSL-CM6A-LR.r4i1p1f1*
tas.historical.IPSL-CM6A-LR.r5i1p1f1*
tas.historical.IPSL-CM6A-LR.r6i1p1f1*
tas.historical.IPSL-CM6A-LR.r7i1p1f1*
tas.historical.IPSL-CM6A-LR.r8i1p1f1*
tas.historical.IPSL-CM6A-LR.r9i1p1f1*
tas.ssp126.IPSL-CM6A-LR.r1i1p1f1*
tas.ssp126.IPSL-CM6A-LR.r2i1p1f1*
tas.ssp126.IPSL-CM6A-LR.r3i1p1f1*
tas.ssp245.IPSL-CM6A-LR.r1i1p1f1*
tas.ssp245.IPSL-CM6A-LR.r2i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r10i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r1i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r2i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r3i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r4i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r5i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r6i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r7i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r8i1p1f1*
tas.ssp370.IPSL-CM6A-LR.r9i1p1f1*
tas.ssp585.IPSL-CM6A-LR.r1i1p1f1*
tas.historical.MIROC-ES2L.r1i1p1f2*
tas.historical.MIROC-ES2L.r2i1p1f2*
tas.historical.MIROC-ES2L.r3i1p1f2*
tas.ssp126.MIROC-ES2L.r1i1p1f2*
tas.ssp245.MIROC-ES2L.r1i1p1f2*
tas.ssp370.MIROC-ES2L.r1i1p1f2*
tas.ssp585.MIROC-ES2L.r